# Overview
Steps to build the network:

1. Load the training data and do a train/validation split.
2. Preprocess data.
3. Build a convolutional neural network to classify traffic signs.
4. Build a feedforward neural network to classify traffic signs.
5. Evaluate performance of final neural network on testing data.

Keep an eye on the network’s accuracy over time. Once the accuracy reaches the 98% range, you can be confident that you’ve built and trained an effective model.

In [212]:
import pandas as pd
import pickle
import numpy as np
from sklearn.model_selection import train_test_split
import math

from keras.models import Sequential
from keras.layers import Dense, Activation

import json

Load the Data

In [214]:
# Load the data by reading the logfile

data = pd.read_csv('data/data/driving_log.csv',dtype={'center': str, 'left':str,'right':str,'steering': np.float32,'throttle': np.float32,'brake': np.float32,'speed': np.float32})
                   #converters={'category',"Price":int} dtype='category')
print(data.dtypes)

X_train = data['center']
y_train = data['steering']

print('Training data size = ', len(X_train))
print('Training labels size = ',len(y_train))
print('Data loaded')

center       object
left         object
right        object
steering    float32
throttle    float32
brake       float32
speed       float32
dtype: object
Training data size =  8036
Training labels size =  8036
Data loaded


In [215]:
# preprocess all data and save to pickle files
from PIL import Image
import matplotlib.pyplot as plt
import pickle
    
def normalise(im):
    width,height = im.size
    im = np.array(im.resize((round(width*0.5),round(height*0.5)), Image.ANTIALIAS)) # reduce to half size
    im = im/255 - 0.5 # normalise data
    return im
        
def append_pickle (pickle_filename, data):
    with open(pickle_filename,'ab') as wfp:
        pickle.dump(data, wfp)

def process_images (data, pickle_filename):
    for filename in data:
        #print('data/data/'+filename)
        im = Image.open('data/data/'+filename)
        width,height = im.size
        #print(width,',',height)
        #plt.imshow(im);
        #plt.show()
        im = normalise(im)
        append_pickle(pickle_filename, im)
    print('Data saved to pickle file:',pickle_filename)
    
def process_labels (data, pickle_filename):
    append_pickle(pickle_filename, data)
    print('Data saved to pickle file:',pickle_filename)

process_images (X_train[1:6],'data.p') # note: here images are saved one by one
process_labels (y_train[1:6],'labels.p') # note: labels are saved all in one go

Data saved to pickle file: data.p
Data saved to pickle file: labels.p


In [216]:
# load pickled data which was written in batches
X_train_t = []
y_train_t = []

def read_pickle_file(filename):
    data_read = []
    if os.path.exists(filename):
        # "with" statements are very handy for opening files. 
        with open(filename,'rb') as rfp:
            while 1:
                if len(data_read)<1:
                    data_read = pickle.load(rfp)
                else:
                    try:
                        data_read = data_read + pickle.load(rfp)
                    except EOFError:
                        break # no more data in the file
        print('Data loaded')    
    else:
        print('ERROR - File not found!')
    return data_read

X_train_t = read_pickle_file('data.p')
y_train_t = read_pickle_file('labels.p')

Data loaded
Data loaded


In [220]:
print(np.size(X_train_t))
print(len(y_train_t))

38400
5


In [114]:
from sklearn.model_selection import train_test_split

# split data into train, validate and test data
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)
print('Data splitted into train and validation data')
print('X_train contains just the path at the moment')
print('X_val contains just the path at the moment')

Data splitted into train and validation data
X_train contains just the path at the moment
X_val contains just the path at the moment


In [115]:
print('Training data size = ', len(X_train))
print('Training labels size = ',len(y_train))
print('Validation data size = ',len(X_val))
print('Validation labels size = ',len(X_val))

Training data size =  6428
Training labels size =  6428
Validation data size =  1608
Validation labels size =  1608


In [18]:
from random import shuffle
batch_size = 10
x = [i for i in range(len(X_val))]
shuffle(x)
indices = x[0:batch_size]
    
print(indices)

[365, 703, 967, 12, 78, 1029, 1552, 421, 474, 1172]


In [74]:
import pickle
train_data = {'data': X_train_im, 'label': y_train }
batch_size = 100
count = 0
while count < (len(train_data)-batch_size):
    with open('train.p','ab') as wfp:
        pickle.dump(train_data[count:count+batch_size], wfp)
    count = count+batch_size
    print('.')
with open('texto','ab') as wfp:
    pickle.dump(train_data[count:len(train_data)], wfp)
    
print('train.p file saved')

val_data = { 'data': X_val_im, 'label': y_val }
count = 0
while count < (len(val_data)-batch_size):
    with open('validation.p','ab') as wfp:
        pickle.dump(val_data[count:count+batch_size], wfp)
    count = count+batch_size
    print('.')
with open('texto','ab') as wfp:
    pickle.dump(val_data[count:len(val_data)], wfp)

print('validation.p file saved')


NameError: name 'X_train_im' is not defined

In [44]:
if os.path.exists(high_scores_filename):
    # "with" statements are very handy for opening files. 
    with open(high_scores_filename,'rb') as rfp:
        while 1:
            try:
                read_data = pickle.load(rfp)
                print(read_data)
            except EOFError:
                break # no more data in the file

('Nino', '10')
('Tom', '20')
('TT', '20')
('OO', '11')


In [20]:
scores = []

In [40]:
import pickle
train_data = { "data": X_train_im, "label": y_train }
pickle.dump( train_data, open( "train.p", "ab" ) )
print('train.p file saved')
val_data = { "data": X_val_im, "label": y_val }
pickle.dump( val_data, open( "validation.p", "wb" ) )
print('validation.p file saved')

MemoryError: 

In [27]:
import pickle
loaded_data = pickle.load( open( "train.p", "rb" ) )
train_X = loaded_data['data']
train_y = loaded_data['label']
print(len(train_X))
print(len(train_y))
loaded_data = pickle.load( open( "validation.p", "rb" ) )
val_X = loaded_data['data']
val_y = loaded_data['label']
print(len(val_X))
print(len(val_y))

6428
6428
1608
1608


In [ ]:
#Load the images from IMG folder and save them

import glob
import cv2
from numpy import array

image_list = []
filename_list = []
for filename in glob.glob('new_treated_images/*.png'):
    filename_list.append(filename)
    im = cv2.imread(filename)
    im = cv2.cvtColor(im,cv2.COLOR_BGR2RGB)
    im = cv2.resize(im,(32,32),interpolation = cv2.INTER_AREA)
    image_list.append(im)

print("There are ",len(image_list),"images in the folder.")
plotImageSet(image_list)

for name in filename_list:
    print(name)

# preprocess the images
image_list_gray = []
for image in image_list:
    image_list_gray.append((rgb2gray(image)-128)/128,);

plotImageSet(image_list_gray,'gray')

In [ ]:
# define model

model = Sequential()

model.add(Dense(32, input_dim=784))
model.add(Activation('relu'))


In [ ]:
# for a multi-input model with 10 classes:

left_branch = Sequential()
left_branch.add(Dense(32, input_dim=784))

right_branch = Sequential()
right_branch.add(Dense(32, input_dim=784))

merged = Merge([left_branch, right_branch], mode='concat')

model = Sequential()
model.add(merged)
model.add(Dense(10, activation='softmax'))

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


# since the model has 2 inputs
model.fit([data_1, data_2], labels, nb_epoch=10, batch_size=32)

In [ ]:
# convert model to json format
json_string = model.to_json()
model = model_from_json(json_string)

#save model to files
model.save_weights(model.h5)
with open('model.json', 'w') as outfile:
    json.dump(model, outfile)